In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import stats, optimize

In [ ]:
# use max likelihood
def neg_likelihood(theta, t, T):
    est_mean = theta[0] + t*theta[1] + t**2*theta[2] + t**3*theta[3] + t**4*theta[4]
    est_std = np.maximum(theta[-3] + t*theta[-2] + t**2*theta[-1], 1e-10)
    return -sum(stats.norm.logpdf(T, loc=est_mean, scale=est_std))

def fit(t, T):
    theta_init = np.array([1,0,0,0,0,1,0,0])
    result = optimize.minimize(neg_likelihood, theta_init, args=(t, T))
    theta = result.x
    result = optimize.minimize(neg_likelihood, theta, args=(t, T))
    theta = result.x
    return theta

def predict(t, theta):
    std = np.maximum(theta[-3] + t*theta[-2] + t**2*theta[-1], np.array([1e-10]))
    mean = theta[0] + t*theta[1] + t**2*theta[2] + t**3*theta[3] + t**4*theta[4]
    return mean, std

In [ ]:
# import the data
training_data = np.loadtxt('Data/MeasuredData_Task4.txt')
# just using right portion for now
time_r = training_data[61:,0].reshape(-1,1)
temp_r = training_data[61:,1].reshape(-1,1)

test_time = np.loadtxt('Data/TestingDataA_Task4.txt').reshape(-1,1)
test_time_r = test_time[60:]

# normalize data
scale_time_r = MinMaxScaler()
scale_temp_r = MinMaxScaler()
time_r = scale_time_r.fit_transform(time_r)
test_time_r = scale_time_r.transform(test_time_r)
temp_r = scale_temp_r.fit_transform(temp_r)

theta_r = fit(time_r, temp_r)
mean_r, std_r = predict(test_time_r, theta_r)

time_r = scale_time_r.inverse_transform(time_r)
test_time_r = scale_time_r.inverse_transform(test_time_r)
temp_r = scale_temp_r.inverse_transform(temp_r)

upper_r = mean_r+std_r
lower_r = mean_r-std_r
mean_r = scale_temp_r.inverse_transform(mean_r)
upper_r = scale_temp_r.inverse_transform(upper_r)
lower_r = scale_temp_r.inverse_transform(lower_r)

plt.scatter(time_r, temp_r)
plt.plot(test_time_r, mean_r)
plt.plot(test_time_r, upper_r)
plt.plot(test_time_r, lower_r)
plt.show()

In [ ]:
time_l = training_data[:61,0].reshape(-1,1)
temp_l = training_data[:61,1].reshape(-1,1)
test_time_l = test_time[:60]

# normalize data
scale_time_l = MinMaxScaler()
scale_temp_l = MinMaxScaler()
time_l = scale_time_l.fit_transform(time_l)
test_time_l = scale_time_l.transform(test_time_l)
temp_l = scale_temp_l.fit_transform(temp_l)

theta_l = fit(time_l, temp_l)
mean_l, std_l = predict(test_time_l, theta_l)

time_l = scale_time_l.inverse_transform(time_l)
test_time_l = scale_time_l.inverse_transform(test_time_l)
temp_l = scale_temp_l.inverse_transform(temp_l)

upper_l = mean_l+std_l
lower_l = mean_l-std_l
mean_l = scale_temp_l.inverse_transform(mean_l)
upper_l = scale_temp_l.inverse_transform(upper_l)
lower_l = scale_temp_l.inverse_transform(lower_l)

plt.scatter(time_l, temp_l)
plt.plot(test_time_l, mean_l)
plt.plot(test_time_l, upper_l)
plt.plot(test_time_l, lower_l)
plt.show()

In [ ]:
# collect output and save
std_l = upper_l - mean_l
std_r = upper_r - mean_r
std = np.concatenate((std_l, std_r), 0)

plt.scatter(test_time, std)
plt.show()

output = np.concatenate((test_time, std), 1)
#np.savetxt('Task4a.txt', output)